# Pneumonia Detection from Chest X-Rays
**Author:** ATHMANE Mohamed Anis  
**Domain:** Medical Data Science / Computer Vision  
**Model:** VGG16 (Transfer Learning)

## 🎯 Project Objective
The goal of this project is to build a Deep Learning model capable of detecting **Pneumonia** from chest X-Ray images.
In a medical context, **False Negatives** (failing to detect a sick patient) are critical errors. Therefore, our primary metric for success is **Recall (Sensitivity)**, aiming to minimize missed diagnoses while maintaining high precision.

## 🛠️ Methodology
1.  **Data Preprocessing:** Rescaling and Data Augmentation to prevent overfitting.
2.  **Architecture:** Transfer Learning using **VGG16** (pre-trained on ImageNet) with a custom classification head.
3.  **Training:** Optimized using `Adam`, with `EarlyStopping` and `ModelCheckpoint` to ensure stability.
4.  **Evaluation:** Rigorous testing on an unseen **Test Set** (624 images).

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## 1. Data Preprocessing & Augmentation
We use **ImageDataGenerator** to load and preprocess the data.
* **Rescaling:** Pixel values are normalized to the [0, 1] range.
* **Data Augmentation (Train only):** To make the model robust to variations in patient positioning, we apply random **rotations**, **zooms**, and **horizontal flips**.
* **Validation Split:** 20% of the training data is reserved for internal validation during training.

In [2]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=20,
    validation_split=0.2
)

train_data = train_gen.flow_from_directory(
    directory='./drive/MyDrive/DATA/train',
    target_size=[224, 224],
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    subset='training'
)

validation_data = train_gen.flow_from_directory(
    directory='./drive/MyDrive/DATA/train',
    target_size=[224, 224],
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    subset='validation'
)

Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.


In [3]:
test_gen = ImageDataGenerator(
    rescale=1./255
)
test_data = test_gen.flow_from_directory(directory='./drive/MyDrive/DATA/test',
    target_size=[224, 224],
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True
)

Found 624 images belonging to 2 classes.


## 2. Model Architecture: Transfer Learning with VGG16
Building a CNN from scratch on a limited medical dataset often leads to overfitting.
Instead, we use **Transfer Learning**:
1.  **Base Model:** VGG16 (weights from ImageNet). We **freeze** these layers to preserve the feature extraction capabilities (edges, textures).
2.  **Custom Head:** We add a `GlobalAveragePooling2D` layer followed by a `Dense` layer and `Dropout` (to further reduce overfitting).
3.  **Output:** A single neuron with `sigmoid` activation for binary classification (Normal vs. Pneumonia).

In [4]:
def create_model():
    vgg = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

    for layer in vgg.layers:
        layer.trainable = False

    out = vgg.output

    x = Flatten()(out)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=vgg.input, outputs=x)

    model.compile(
        loss='binary_crossentropy',
        optimizer='Adam',
        metrics=['accuracy', Recall(name='recall'), Precision(name='precision')]
    )

    return model
my_model = create_model()
my_model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │        25,089 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,739,777 (56.23 MB)

 Trainable params: 25,089 (98.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

## 3. Training Configuration
We compile the model using the **Adam** optimizer with a low learning rate (`1e-3`) to fine-tune the weights without destroying the pre-learned features.

**Callbacks used:**
* `EarlyStopping`: Stops training if validation loss doesn't improve for 3 epochs.
* `ModelCheckpoint`: Saves only the best model based on **Validation Recall**, ensuring we keep the version that misses the fewest cases.

In [5]:
checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_recall',
    save_best_only=True,
    mode='max'
)
early_stop = EarlyStopping(
    monitor='val_recall',
    patience=3,
    mode='max',
    restore_best_weights=True
)

In [6]:
history = my_model.fit(
    train_data,
    validation_data=validation_data,
    epochs=20,
    callbacks=[early_stop, checkpoint]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 776s 6s/step - accuracy: 0.8196 - loss: 0.3803 - precision: 0.8720 - recall: 0.8835 - val_accuracy: 0.9262 - val_loss: 0.1763 - val_precision: 0.9915 - val_recall: 0.9084
Epoch 2/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 138s 1s/step - accuracy: 0.9476 - loss: 0.1396 - precision: 0.9661 - recall: 0.9645 - val_accuracy: 0.9425 - val_loss: 0.1422 - val_precision: 0.9838 - val_recall: 0.9381
Epoch 3/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.9539 - loss: 0.1266 - precision: 0.9684 - recall: 0.9691 - val_accuracy: 0.9597 - val_loss: 0.1018 - val_precision: 0.9854 - val_recall: 0.9600
Epoch 4/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.9585 - loss: 0.1188 - precision: 0.9734 - recall: 0.9714 - val_accuracy: 0.9540 - val_loss: 0.1131 - val_precision: 0.9840 - val_recall: 0.9535
Epoch 5/20
131/131 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.9635 - loss: 0.1006 - precision: 0.9763 - recall: 0.9733 - val_accuracy: 0.9521 - val_los

## 4. Final Evaluation on Test Set
The model is now evaluated on the **Test Set** (624 images) which it has **never seen** during training. This provides an unbiased measure of real-world performance.

In [ ]:
my_model.load_weights('models/best_model.keras')

print("--- Final results on the Test Set ---")
test_loss, test_acc, test_recall, test_precision = my_model.evaluate(test_data)

--- Final results on the Test Set ---
20/20 ━━━━━━━━━━━━━━━━━━━━ 261s 13s/step - accuracy: 0.9063 - loss: 0.2831 - precision: 0.8840 - recall: 0.9811


In [11]:
print(f"Recall Final : {test_recall*100:.2f}%")

Recall Final : 98.46%


In [12]:
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_acc}')
print(f"Test Recall: {test_recall}")
print(f"Test Precision: {test_precision}")

Test Loss: 0.28089284896850586
Test Accuracy: 0.9102563858032227
Test Recall: 0.9846153855323792
Test Precision: 0.8847926259040833


# 🏆 Conclusion & Results

The model achieved excellent performance on the unseen Test Set:

| Metric | Score | Clinical Interpretation |
| :--- | :--- | :--- |
| **Recall** | **98.46%** | **Critical Success.** The model detects almost all positive pneumonia cases. |
| **Accuracy** | **91.02%** | The model is generally correct 9 times out of 10. |
| **Precision** | **88.48%** | The False Positive rate is within an acceptable range for a screening tool. |

